In [18]:
# For label
import pandas as pd
import numpy as np

np.quantile([1,4,5,7,10], 0.9)

8.8

In [15]:
import sys
import torch
import datasets
import evaluate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

sys.path.append("../")

In [2]:
from llmsearch.utils.model_utils import seed_everything, get_device

In [3]:
seed = 42

device = get_device()
print(f"Device : {device}")

Device : mps


In [4]:
model_id = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

In [5]:
dataset = datasets.load_dataset("samsum")
rouge_metric = evaluate.load("rouge")

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
import nltk
from typing import List, Dict

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)

    return result


def infer_single_sample(
    model, input_text: str, model_input_tokenizer_kwargs: Dict, generation_kwargs: Dict
):
    gc.collect()
    start = time.time()
    input_ids = tokenizer(
        text=input_text, **model_input_tokenizer_kwargs, return_tensors="pt"
    ).input_ids.to(device)
    output_ids = model.generate(input_ids, **generation_kwargs)[0]
    decoded_output = tokenizer.decode(token_ids=output_ids, skip_special_tokens=True)
    end = time.time()
    latency = (end - start) * 1000
    return decoded_output, latency

In [7]:
import time

import gc
from tqdm.auto import tqdm

def get_metrics(
    model, prompt, dataset, model_input_tokenizer_kwargs, generation_kwargs
):
    preds = []
    gts = []

    for item in tqdm(samples_to_tune_on):
        model_input = prompt.format(input_text=item["dialogue"])
        output, _ = infer_single_sample(
            model, model_input, model_input_tokenizer_kwargs, generation_kwargs
        )
        preds.append(output)
        gts.append(item["summary"])
        # print(f"Model Output : {output}")
        # print(f"Label : {item['summary']}")
        # print()
        # print("===" * 10)

    metric = get_rouge_score(y_true=gts, y_pred=preds)
    return metric, preds, gts


# take 1st 100 samples
sample_size = 10
samples_to_tune_on = datasets.Dataset.from_dict(dataset["validation"][:sample_size])

model_input_tokenizer_kwargs = {
    "max_length": 1000,
    "truncation": True,
    "padding": False,
}
prompt = "{input_text} Summarize:"

generation_kwargs = {
    "temperature": 1,
    "max_new_tokens": 100,
}

# Check metric on these examples before any generation param tuning

metric, _, _ = get_metrics(
    model=model,
    prompt=prompt,
    dataset=samples_to_tune_on,
    model_input_tokenizer_kwargs=model_input_tokenizer_kwargs,
    generation_kwargs=generation_kwargs,
)
print(metric)

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-vDz_II8E-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:2256: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0 or stopping_criteria(input_ids, scores):


{'rouge1': 0.3273997872425861, 'rouge2': 0.1337747216967607, 'rougeL': 0.28957710411916593, 'rougeLsum': 0.31089543620393756}


In [11]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from llmsearch.wrappers.estimator import EstimatorWrapper
from llmsearch.tuner import Tuner


hyp_param_grid = {
    "max_new_tokens": [200],
    "num_beams": [3],
    "temperature": [1],
    "epsilon_cutoff": [3e-4, 6e-4, 9e-4, 0],
    "repetition_penalty": [0.3, 0.7, 1.0],
    "no_repeat_ngram_size": [0, 2, 3, 4],
}

model_input_tokenizer_kwargs = {
    "max_length": 1000,
    "truncation": True,
    "padding": True,
}

tuner_ob = Tuner(
    model = model,
    tokenizer = tokenizer,
    device = device,
    batch_size = 32,
    model_input_tokenizer_kwargs=model_input_tokenizer_kwargs,
    dataset = (samples_to_tune_on['dialogue'], samples_to_tune_on['summary'])
)

def my_metric(y_true: List, y_pred: List):
    def postprocess_text(preds: List, labels: List):
        """Postprocess text(preds & labels) to rouge format for evaluation"""
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
        return preds, labels

    """Get rouge score from y_true & y_pred"""
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)

    # TODO : fix metric
    return result["rouge2"]

In [12]:
scorer = make_scorer(score_func=my_metric, greater_is_better=True)

clf = RandomizedSearchCV(
    estimator=tuner_ob.estimator,
    param_distributions=hyp_param_grid,
    n_iter=30,
    scoring=scorer,
    cv=5,
    random_state=seed,
    n_jobs=1,
    verbose=1,
)

In [13]:
clf.fit(X=samples_to_tune_on["dialogue"], y=samples_to_tune_on["summary"])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-vDz_II8E-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:686: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RandomizedSearchCV(cv=5,
                   estimator=EstimatorWrapper(batch_size=32, device='mps',
                                              model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False...
                                              tokenizer=T5TokenizerFast(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>', '<extra_id_58>', '<extra_id_59>', '<extra_id_60>', '<extra_id_61>', '<extra_id_62>', '<extra_id_63>', '<extra_id_64>', '<extra_id_65>', '<extra_id_66>', '<extra_id_67>', '<extra_id_68>', '<extra_id_69>', '<extra_id_70>', '<extra_id_71>', '<extra_id_72>', '<extra_id_73>', '<extra_id_74>', '<extra_id_75>', '<extra_id_76>', '<extra_id_77>', '<extra_id_78>', '<extra_id_79>', '<extra_id_80>', '<extra_id_81>', '<extra_id_82>', '<extra_id_83>', '<extra_id_84>', '<extra_id_85>', '<extra_id_86>', '<extra_id_87>', '<extra_id_88>', '<extra_id_89>', '<extra_id_90>', '<extra_id_91>', '<extra_id_92>', '<extra_id_93>', '<extra_id_94>', '<extra_id_95>', '<extra_id_96>', '<extra_id_97>', '<extra_id_98>', '<extra_id_99>']})),
                   n_iter=30, n_jobs=1,
                   param_distributions={'epsilon_cutoff': [0.0003, 0.0006,
                                                           0.0009, 0],
                                        'max_new_tokens': [200],
                                        'no_repeat_ngram_size': [0, 2, 3, 4],
                                        'num_beams': [3],
                                        'repetition_penalty': [0.3, 0.7, 1.0],
                                        'temperature': [1]},
                   random_state=42, scoring=make_scorer(my_metric), verbose=1)

In [14]:
clf.best_params_

{'temperature': 1,
 'repetition_penalty': 1.0,
 'num_beams': 3,
 'no_repeat_ngram_size': 2,
 'max_new_tokens': 200,
 'epsilon_cutoff': 0.0006}